# Web-scraping to collect publicly available newspaper

In [ ]:
import os
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Specify the path to the Chrome WebDriver executable
chrome_driver_path = "C:/Users/claudioq/Desktop/chrome-win64/chrome.exe"

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = "/path/to/chrome"

# Set the executable path as an option
chrome_options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")

for newspaper in ["Javan"]: # , "JaameJam", "KayhanNews", "Etemaad", "TejaratOnline"] :

    for month in np.arange(1, 13):

        for day in np.arange(1, 31):
            try:
                # Set up your WebDriver (e.g., for Chrome) with the options
                driver = webdriver.Chrome(options=chrome_options)

                # Navigate to the initial URL
                if (month<10)&(day<10) :
                    initial_url = f"https://www.pishkhan.com/pdfviewer.php?paper="+newspaper+f"&date=13980{month}0{day}"
                elif (month<10) :
                    initial_url = f"https://www.pishkhan.com/pdfviewer.php?paper="+newspaper+f"&date=13980{month}{day}"
                elif (day<10) :
                    initial_url = f"https://www.pishkhan.com/pdfviewer.php?paper="+newspaper+f"&date=1398{month}0{day}"
                else:
                    initial_url = f"https://www.pishkhan.com/pdfviewer.php?paper="+newspaper+f"&date=1398{month}{day}"
                                
                driver.get(initial_url)

                # Wait for the redirect to occur
                wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed

                # Wait until the URL changes
                wait.until(EC.url_changes(initial_url))

                # Once redirected, capture the current URL
                pdf_url = driver.current_url

                # Verify that the URL ends with ".pdf" to ensure it's the PDF file
                if pdf_url.endswith(".pdf"):
                    print(f"PDF URL: {pdf_url}")

                    # Extract the filename from the URL
                    filename = pdf_url.split("/")[-1]

                    # Define the download path and ensure it exists
                    download_path = "C:/Users/claudioq/Desktop/newspapers/"
                    os.makedirs(download_path, exist_ok=True)

                    # Define the full path to save the PDF
                    save_path = os.path.join(download_path, newspaper+f"1398" + initial_url[-4:] + ".pdf")

                    # You can download the PDF here using your preferred method
                    # For example, you can use requests to download it:
                    import requests

                    response = requests.get(pdf_url, stream=True)
                    if response.status_code == 200:
                        with open(save_path, "wb") as pdf_file:
                            for chunk in response.iter_content(chunk_size=1024):
                                pdf_file.write(chunk)
                        print(f"PDF downloaded and saved as '{save_path}' successfully.")
                    else:
                        print("Failed to download the PDF.")

            except Exception as e:
                print(f"Error: {e}")

            finally:
                # Don't forget to close the WebDriver when done
                if 'driver' in locals():
                    driver.quit()

PDF URL: https://www.pishkhan.com/Archive/1398/01/13980117/Javan7497110971184949551958.pdf
PDF downloaded and saved as 'C:/Users/claudioq/Desktop/newspapers/Javan13980117.pdf' successfully.
PDF URL: https://www.pishkhan.com/Archive/1398/01/13980118/Javan7497110971184949561959.pdf
PDF downloaded and saved as 'C:/Users/claudioq/Desktop/newspapers/Javan13980118.pdf' successfully.
PDF URL: https://www.pishkhan.com/Archive/1398/01/13980119/Javan7497110971184949571959.pdf
PDF downloaded and saved as 'C:/Users/claudioq/Desktop/newspapers/Javan13980119.pdf' successfully.
PDF URL: https://www.pishkhan.com/Archive/1398/01/13980120/Javan7497110971185049482060.pdf
PDF downloaded and saved as 'C:/Users/claudioq/Desktop/newspapers/Javan13980120.pdf' successfully.
PDF URL: https://www.pishkhan.com/Archive/1398/01/13980121/Javan7497110971185049492060.pdf
PDF downloaded and saved as 'C:/Users/claudioq/Desktop/newspapers/Javan13980121.pdf' successfully.
PDF URL: https://www.pishkhan.com/Archive/1398/01/

# Convert the files to text data using OCR when necessary

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

""" Initialize the Output dataframes """
result = np.zeros(shape=(365*8,6))
result = pd.DataFrame(result, columns=["year","month","day","newspaper","word","OCR"])


# Specify the path to the folder you want to list files from
folder_path = "C:/Users/claudioq/Desktop/newspapers/"

# Use the os.listdir() function to get a list of file names in the folder
list_of_newspapers = os.listdir(folder_path)

# list_of_newspapers

download_path = "C:/Users/claudioq/Desktop/newspapers/"

# define key terms
# Define the target word you want to search for
target_word = ['بنزین'
              , 'نیزنب'
              , 'بنزن'
              ]

enum = 0
for month in np.arange(2, 13):
    if month<10:
        month = f"0{month}"
    else:
        month = f"{month}"
    
    for day in np.arange(1, 31):
        if day<10:
            day = f"0{day}"
        else:
            day = f"{day}"
            
        for newspaper in ["JaameJam", "Javan", "KayhanNews", "Etemaad", "TejaratOnline", "Iran", "khorasannews", "Shahrvand"] :

            path = newspaper+"1398"+month+day+".pdf"
            
            if path not in list_of_newspapers:
                continue
            
            # Open the PDF file
            pdf_file_path = download_path+path
            pdf_document = fitz.open(pdf_file_path)

            # Initialize a counter for the word occurrences
            word_count = 0

            # Iterate through each page in the PDF
            text = ""
            for page_number in range(pdf_document.page_count):
                # Get the text content of the page
                page = pdf_document[page_number]
                page_text = page.get_text()

                # If the page_text is empty or contains unrecognized characters, perform OCR
                if not page_text or '��' in page_text:
                    try:
                        result.iloc[enum,5] = 1
                        image = page.get_pixmap()
                        image_data = Image.frombytes("RGB", [image.width, image.height], image.samples)
                        page_text = pytesseract.image_to_string(image_data, lang='fas')
                    except:
                        result.iloc[enum,5] = -1
                
                text = text + page_text
                # Count the occurrences of the target word in the page text
                for t in target_word:
                    word_count += page_text.count(t)
            
            # Close the PDF document
            pdf_document.close()
            # save the text file
            try:
                with open(download_path+ "Text/" +newspaper+"1398"+month+day+".txt", 'w', encoding="utf-8") as f:
                    f.write(text)
            except:
                print("failed to save the text file"+newspaper+"1398"+month+day+".txt")
            
            # print(f'The word "{target_word}" appears {word_count} times in the PDF.')
            
            result.iloc[enum,0] = 1398
            result.iloc[enum,1] = month
            result.iloc[enum,2] = day
            result.iloc[enum,3] = newspaper
            result.iloc[enum,4] = word_count
            enum=enum+1
            

    result.to_csv(download_path+"a_result.csv")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    print("next month finished")
    



Current Time = 20:33:07
next month finished
Current Time = 21:41:28
next month finished
Current Time = 22:58:39
next month finished
Current Time = 00:14:07
next month finished
Current Time = 00:51:19
next month finished
Current Time = 02:01:23
next month finished
Current Time = 03:01:31
next month finished
Current Time = 04:14:23
next month finished
Current Time = 05:20:47
next month finished
Current Time = 06:31:59
next month finished
Current Time = 07:31:35
next month finished


# Text analysis: word counts

In [ ]:
""" Initialize the Output dataframes """
result = np.zeros(shape=(365*8,6))
result = pd.DataFrame(result, columns=["year","month","day","newspaper","covid","gasoline"])

download_path = "C:/Users/claudioq/Desktop/newspapers/"
# Use the os.listdir() function to get a list of file names in the folder
list_of_newspapers = os.listdir(download_path+"Text/")


# define key terms
# Define the target word you want to search for
covid_text = ['کرونا'
              , 'کووید'
              , 'انورک'
              , 'دیووک'             
              ]

# covid_text = ['سهمیه بندی'
#               , 'یدنب هیمهس']

gasoline_text = ['بنزین'
              , 'نیزنب'
              , 'بنزن'
              ]

enum = 0
for month in np.arange(1, 13):
    if month<10:
        month = f"0{month}"
    else:
        month = f"{month}"
    
    for day in np.arange(1, 31):
        if day<10:
            day = f"0{day}"
        else:
            day = f"{day}"
            
        for newspaper in ["JaameJam", "Javan", "KayhanNews", "Etemaad", "TejaratOnline", "Iran", "khorasannews", "Shahrvand"] :

            path = newspaper+"1398"+month+day+".txt"
            
            if path not in list_of_newspapers:
                continue

            # Iterate through each page in the PDF
            pdf_file_path = download_path+"Text/"+path
#             with open(pdf_file_path) as f:
#                 text = f.read()
            with open(pdf_file_path , 'r', encoding='utf-8') as file:
                # Read the content of the file
                text = file.read()
                
            # Count the occurrences of the target word in the page text
            # Initialize a counter for the word occurrences
            covid = 0
            gasoline = 0
            for t in covid_text:
                covid += text.count(t)
            for t in gasoline_text:
                gasoline += text.count(t)
            
            file.close()
            
            result.iloc[enum,0] = 1398
            result.iloc[enum,1] = month
            result.iloc[enum,2] = day
            result.iloc[enum,3] = newspaper
            result.iloc[enum,4] = covid
            result.iloc[enum,5] = gasoline
            enum=enum+1
            

#     result.to_csv(download_path+"a_result.csv")
result.to_csv(download_path+"covid.csv")


In [40]:
result

,year,month,day,newspaper,covid,gasoline
0,1398.0,02,01,Javan,0.0,0.0
1,1398.0,02,01,KayhanNews,0.0,0.0
2,1398.0,02,01,Iran,0.0,2.0
3,1398.0,02,01,khorasannews,0.0,1.0
4,1398.0,02,01,Shahrvand,0.0,5.0
...,...,...,...,...,...,...
2915,0.0,0,0,0,0.0,0.0
2916,0.0,0,0,0,0.0,0.0
2917,0.0,0,0,0,0.0,0.0
2918,0.0,0,0,0,0.0,0.0
